### Get the packages ready for usage

In [1]:
import urllib.request, urllib.error, urllib.parse
import re
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

### Prepare the DataFrame

#### Getting the feel of the data

In [2]:
url_sample= 'https://www.peakware.com/peaks.php?pk=1001'

In [5]:
df = pd.read_html(url_sample)

peak_sample = pd.DataFrame(df[0])

peak_sample

,0,1
0,Elevation (feet):,9415
1,Elevation (meters):,2869
2,Continent:,North America
3,Country:,United States
4,Range/Region:,Cascade Range
5,State:,Washington
6,Latitude:,47.475158
7,Longitude:,-120.902395
8,Difficulty:,Technical Climb
9,Best months for climbing:,"Jun, Jul, Aug"


In [6]:
# checking the conversion of dataframe read from the html
# to a dictionary which will later be stored properly in another dataframe
init_dict={}
for index in range(len(peak_sample)):
    k,v=peak_sample.iloc[index]
    init_dict[k]=[v]

init_dict

{'Best months for climbing:': ['Jun, Jul, Aug'],
 'Continent:': ['North America'],
 'Convenient Center:': ['Leavenworth, Washington'],
 'Country:': ['United States'],
 'Difficulty:': ['Technical Climb'],
 'Elevation (feet):': ['9415'],
 'Elevation (meters):': ['2869'],
 'First successful climber(s):': ['A. McPherson and party'],
 'Latitude:': ['47.475158'],
 'Longitude:': ['-120.902395'],
 'Nearest major airport:': ['Seattle, Washington'],
 'Range/Region:': ['Cascade Range'],
 'State:': ['Washington'],
 'Year first climbed:': ['1873']}

### Retrieve the Peak's Name

#### a much better way was realised with BeautifulSoup in the attached file webscraping_peakware

In [8]:
# Get site text
with urllib.request.urlopen(url_sample) as f:
    site_text=(f.read().decode('utf-8'))

In [9]:
# e.g. peak no. 1001 can be identified by the string
# <h1>Mount Stuart</h1>
start_str = '<h1>'
end_str = '</h1>'
start_index = site_text.find(start_str)
end_index = site_text.find(end_str) 

peak_name = site_text[(start_index+len(start_str)):end_index]
print(peak_name)

Mount Stuart


In [10]:
# checking the dataframe created from the dictionary created earlier 
init_df = pd.DataFrame(data=init_dict, index=[peak_name])
init_df

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Nearest major airport:,Range/Region:,State:,Year first climbed:
Mount Stuart,"Jun, Jul, Aug",North America,"Leavenworth, Washington",United States,Technical Climb,9415,2869,A. McPherson and party,47.475158,-120.902395,"Seattle, Washington",Cascade Range,Washington,1873


### Write DataFrame to a file for future use

In [11]:
# saving it to a skeleton data file
init_df.to_csv('Peak_db.csv')

### The making of Peak's DataFrame

In [ ]:
# The code below works fine and completed webscraping the data
# without a problem.
# however, a better version (webscraping_peakware) I wrote in
# PyCharm is attached. The py code also completed successful webscraping
# of all data in peakware.

In [2]:
# This is a Progress Bar I found on the internet
# because once the webscraping began there was no indication of the progress
from ipywidgets import FloatProgress
from IPython.display import display

In [3]:
# retrieve the DataFrame skeleton
Peak_df = pd.read_csv('Peak_db.csv', index_col=0)
Peak_df

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Nearest major airport:,Range/Region:,State:,Year first climbed:
Mount Stuart,"Jun, Jul, Aug",North America,"Leavenworth, Washington",United States,Technical Climb,9415,2869,A. McPherson and party,47.475158,-120.902395,"Seattle, Washington",Cascade Range,Washington,1873


In [2]:
progress = FloatProgress(min=0, max=5000,
                         description='Web scraping progress',
                    bar_style='success')
display(progress)
# make a loop that scans the site peak after peak
# based on webpage serial number
for peak_num in range(1,5000):
    peak_data_dict={} # initialising temp data dict
    peak_url = 'https://www.peakware.com/peaks.php?pk={}'.format(peak_num)
    # Get site text
    with urllib.request.urlopen(peak_url) as f:
        site_text=(f.read().decode('utf-8','ignore'))
    # Get the peak's name from site text
    start_str = '<h1'
    end_str = '</h1>'
    start_index = site_text.find(start_str)
    end_index = site_text.find(end_str)
    peak_name = site_text[(start_index+len(start_str)+1):end_index]
    # exit loop when no more peaks
    if peak_name[-9:-1:1] == 'Not Foun':
        pass
    else:
        progress.value += 1
        #print(peak_num, peak_name, end=' ')
        # building a dict with peak's data
        df = pd.read_html(peak_url)
        peak_sample = pd.DataFrame(df[0])
        for index in range(len(peak_sample)):
            k,v=peak_sample.iloc[index]
            peak_data_dict[k]=[v]
        # add peak name to peak data dict
        Peak_data = pd.DataFrame(data=peak_data_dict, index=[peak_name])
        Peak_df=pd.concat([Peak_df, Peak_data])


In [10]:
# Save DataFrame to a file for future use
Peak_df.to_csv('Peak_temp_db.csv')

In [8]:
Peak_df

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Mount Stuart,"Jun, Jul, Aug",North America,"Leavenworth, Washington",United States,Technical Climb,9415,2869,A. McPherson and party,47.4752,-120.902,NaN,"Seattle, Washington",NaN,Cascade Range,Washington,NaN,1873
Mount Irvine,NaN,North America,Lone Pine,United States,NaN,13914,4241,NaN,36.5558,-118.262,NaN,NaN,NaN,Sierra Nevada,California,NaN,NaN
Peak 7480,"Jan, Feb, Apr, May, Sep, Oct",North America,"Carlsbad, NM.",United States,Walk up,7480,2280,NaN,32,-104.817,NaN,"El Paso, TX.",NaN,Southwest Basins and Ranges,Texas,NaN,NaN
Norton Peak,"Jun, Jul, Aug, Sep, Oct",North America,Ketchum,United States,Scramble,10336,3150,NaN,43.7619,-114.652,NaN,Hailey,NaN,Idaho-Bitterroot Rockies,Idaho,NaN,NaN
Mont Aiguille,"Jul, Aug, Sep",Europe,"Grenoble, France",France,Technical Climb,6842,2085,Antoine de Ville,44.8333,5.55,NaN,"Grenoble or Lyon, France",NaN,Dauphine Alps,NaN,NaN,1492
Aiguille Verte,"Jul, Aug, Sep",Europe,"Chamonix, France",France,Basic Snow/Ice Climb,13524,4122,"Edward Whymper, Christian Almer, Franz Biener",45.9833,6.43333,NaN,"Chamonix, France",NaN,Savoy Alps,NaN,NaN,1865
Mount Alberta,"Jun, Jul, Aug, Sep",North America,"Jasper, Alberta",Canada,Technical Climb,11874,3619,"Yuko Maki and party with , H. Fuhrer, H. Kohle...",52.288008,-117.476463,NaN,"Edmonton or Calgary, Alberta",Alberta,Canadian Rockies,NaN,NaN,1925
Aletschhorn,"Jul, Aug, Sep",Europe,"Bettmeralp or Blatten, Switzerland",Switzerland,Technical Climb,13763,4195,"Francis Tuckett, J.J. Bennen, P. Bohren, Victo...",46.4667,7.98333,NaN,"Bern, Switzerland",NaN,Berner Oberland,NaN,NaN,1859
Alpamayo,"May, Jun, Jul, Aug, Sep",South America,"Huarez, Peru",Peru,Basic Snow/Ice Climb,19511,5947,"G. Hauser, F. Knauss, B. Huhn, H. Wiedmann (Ge...",-8.878203,-77.654371,NaN,"Lima, Peru",NaN,Peruvian Andes,NaN,NaN,1951
Ama Dablam,"Apr, May, Sep, Oct",Asia,"Namche Bazar, Nepal",Nepal,Major Mountain Expedition,22494,6856,"Mike Gill, Barry Bishop, Mike Ward, Wally Romanes",27.861111,86.861111,NaN,"Kathmandu, Nepal",NaN,Central Nepal Himalaya,NaN,NaN,1961


In [9]:
Peak_data

,Best months for climbing:,Continent:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,Latitude:,Longitude:,Range/Region:,Year first climbed:
Gangkar Punsum,"Apr, May, Sep, Oct",Asia,Bhutan/China,Major Mountain Expedition,24836,7570,28.03,90.27,Bhutan Himalaya,UNCLIMBED


### Continue web scraping from where I stopped

In [3]:
# I had to stop webscraping and continue after a while

In [2]:
from ipywidgets import FloatProgress
from IPython.display import display

In [4]:
# retrieve the DataFrame
Peak_df = pd.read_csv('Peak_db.csv', index_col=0)
Peak_df

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Nearest major airport:,Range/Region:,State:,Year first climbed:
Mount Stuart,"Jun, Jul, Aug",North America,"Leavenworth, Washington",United States,Technical Climb,9415,2869,A. McPherson and party,47.475158,-120.902395,"Seattle, Washington",Cascade Range,Washington,1873


In [5]:
progress = FloatProgress(min=2491, max=5000,
                         description='progress',
                    bar_style='success')
display(progress)
# make a loop that scans the site peak after peak
# based on webpage serial number
for peak_num in range(2491,5000):
    peak_data_dict={} # initialising temp data dict
    peak_url = 'https://www.peakware.com/peaks.php?pk={}'.format(peak_num)
    # Get site text
    with urllib.request.urlopen(peak_url) as f:
        site_text=(f.read().decode('utf-8','ignore'))
    # Get the peak's name from site text
    start_str = '<h1'
    end_str = '</h1>'
    start_index = site_text.find(start_str)
    end_index = site_text.find(end_str)
    peak_name = site_text[(start_index+len(start_str)+1):end_index]
    # exit loop when no more peaks
    if peak_name[-9:-1:1] == 'Not Foun':
        pass
    else:
        progress.value += 1
        #print(peak_num, peak_name, end=' ')
        # building a dict with peak's data
        df = pd.read_html(peak_url)
        peak_sample = pd.DataFrame(df[0])
        for index in range(len(peak_sample)):
            k,v=peak_sample.iloc[index]
            peak_data_dict[k]=[v]
        # add peak name to peak data dict
        Peak_data = pd.DataFrame(data=peak_data_dict, index=[peak_name])
        Peak_df=pd.concat([Peak_df, Peak_data])


A Jupyter Widget

In [10]:
# Save DataFrame to a second file for future use
Peak_df.to_csv('Peak_temp2_db.csv')

In [6]:
peak_num

4999

In [7]:
peak_name

'class="left">Peak Not Found'

In [8]:
Peak_data

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Range/Region:,Volcanic status:
Padre Encantado,"Jan, Feb, Jun, Jul, Aug, Dec",South America,Quito,Ecuador,Walk up,15371,4685,-0.171525,-78.576218,2002,Quito,Ecuador Andes,Active


In [9]:
Peak_df

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Mount Stuart,"Jun, Jul, Aug",North America,"Leavenworth, Washington",United States,Technical Climb,9415,2869,A. McPherson and party,47.4752,-120.902,NaN,"Seattle, Washington",NaN,Cascade Range,Washington,NaN,1873
Storsylen,"May, Jun, Jul, Aug, Sep",Europe,Trondheim,Norway,Walk up,5781,1762,NaN,63.0216,12.1997,NaN,"Trondheim airport, Vaernes",NaN,NaN,NaN,NaN,1885
Burney Mountain,NaN,North America,Burney,United States,Walk up,7863,NaN,NaN,40.806569,-121.627931,NaN,Redding,NaN,Cascade Range,California,NaN,NaN
Acho,"Oct, Nov",Asia,Sari,Iran,Walk up,5525,1684,NaN,36.227,53.1633,NaN,Sari Dasht-e Naz Airport,NaN,Alborz,NaN,NaN,NaN
Sucre,"Jan, Jun, Jul, Dec",South America,Papallacta,Ecuador,Scramble,12313,3753,NaN,0.364733,-78.1368,NaN,Quito Ecuador,NaN,Ecuador Andes,NaN,NaN,NaN
Dans Mountain,"Jan, Feb, Mar, Apr, May, Jun, Jul, Aug, Sep, O...",North America,Cumberland,United States,Walk up,2898,883,NaN,39.5814,-78.8975,NaN,Baltimore,NaN,Allegheny Mountains,Maryland,NaN,NaN
The Roaches,"Mar, Apr, May, Jun, Jul, Aug",Europe,Leek,United Kingdom,Scramble,1657,505,NaN,53.171389,-2,NaN,Manchester,NaN,Pennine Chain,NaN,NaN,NaN
Axe Edge,"May, Jun, Jul, Aug, Sep",Europe,Buxton/Leek,United Kingdom,Walk up,1808,551,NaN,53.2325,-1.948889,NaN,Manchester,NaN,Pennine Chain,NaN,NaN,NaN
Mam Tor,"May, Jun, Jul, Aug",Europe,Glossop,United Kingdom,Walk up,1696,517,NaN,53.349167,-1.810556,NaN,Manchester,NaN,Pennine Chain,NaN,NaN,NaN
Gun Hill,"Mar, Apr, May, Jun, Jul, Aug, Sep",Europe,Leek,United Kingdom,Walk up,1224,373,NaN,53.150556,-2.046389,NaN,Manchester,NaN,Pennine Chain,NaN,NaN,NaN


### Merging the saved database

In [5]:
# Load the 2 csv files with the data
# and merge them into 1 dataframe
# retrieve the DataFrame
Peak_df1 = pd.read_csv('Peak_temp_db.csv', index_col=0)
Peak_df2 = pd.read_csv('Peak_temp2_db.csv', index_col=0)
Peak_df = pd.concat([Peak_df1, Peak_df2])#, verify_integrity=True)

In [6]:
# saving to merged dataframe to 1 database as csv file
Peak_df.to_csv('Peak_data_db.csv')

### Cleaning the data

Make sure the dtypes are reasonable, namely numbers are numbers and strings are strings.

Notes:
* The order of the columns is not important.
* The names of the peaks are the index of the DataFrame.

Missing data:
* Not all the peaks have all the features. Put None where the data is missing.
* If only one of the elevations is given, then fill the missing data.
* If both elevations are missing, then drop the peak record.

Data manipulation:
* If a peak is listed with more than a single country, then use the first country.
* If the date of the first climbing is recorded as an irregular date, then try to manipulate it so that only the year data is preserved. If the data is too obscure, then put None instead.
* Note that some terms are written differently for different peaks. Use the lower() method to avoid mistakes.

In [7]:
# read the data
Peak_df = pd.read_csv('Peak_data_db.csv', index_col=0)

In [8]:
Peak_df.head()

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Mount Stuart,"Jun, Jul, Aug",North America,"Leavenworth, Washington",United States,Technical Climb,9415.0,2869.0,A. McPherson and party,47.475158,-120.902395,NaN,"Seattle, Washington",NaN,Cascade Range,Washington,NaN,1873.0
Mount Irvine,NaN,North America,Lone Pine,United States,NaN,13914.0,4241.0,NaN,36.555800,-118.262000,NaN,NaN,NaN,Sierra Nevada,California,NaN,NaN
Peak 7480,"Jan, Feb, Apr, May, Sep, Oct",North America,"Carlsbad, NM.",United States,Walk up,7480.0,2280.0,NaN,32.000000,-104.817000,NaN,"El Paso, TX.",NaN,Southwest Basins and Ranges,Texas,NaN,NaN
Norton Peak,"Jun, Jul, Aug, Sep, Oct",North America,Ketchum,United States,Scramble,10336.0,3150.0,NaN,43.761900,-114.652000,NaN,Hailey,NaN,Idaho-Bitterroot Rockies,Idaho,NaN,NaN
Mont Aiguille,"Jul, Aug, Sep",Europe,"Grenoble, France",France,Technical Climb,6842.0,2085.0,Antoine de Ville,44.833300,5.550000,NaN,"Grenoble or Lyon, France",NaN,Dauphine Alps,NaN,NaN,1492


In [9]:
Peak_df.dtypes

Best months for climbing:        object
Continent:                       object
Convenient Center:               object
Country:                         object
Difficulty:                      object
Elevation (feet):               float64
Elevation (meters):             float64
First successful climber(s):     object
Latitude:                       float64
Longitude:                      float64
Most recent eruption:            object
Nearest major airport:           object
Province:                        object
Range/Region:                    object
State:                           object
Volcanic status:                 object
Year first climbed:              object
dtype: object

### Checking for duplicates

In [10]:
# read the data
Peak_df = pd.read_csv('Peak_data_db.csv', index_col=0)

In [11]:
# check if there are dulicate peaks
peak_duplicates = Peak_df[Peak_df.duplicated()==True]
peak_duplicates

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Sierra De Famatina,"Jan, Feb, Mar, Nov, Dec",South America,Chilecito 83 km (Famatina 50 km),Argentina,Basic Snow/Ice Climb,20505.0,6250.0,NaN,-29.033300,-67.816700,NaN,"La Rioja, 90 km from Chilecito",NaN,Central Argentina-Chile,NaN,NaN,NaN
Mount Stuart,"Jun, Jul, Aug",North America,"Leavenworth, Washington",United States,Technical Climb,9415.0,2869.0,A. McPherson and party,47.475158,-120.902395,NaN,"Seattle, Washington",NaN,Cascade Range,Washington,NaN,1873
Aukstojas,"May, Jun, Jul, Aug, Sep",Europe,Vilnius,Lithuania,Walk up,961.0,293.0,NaN,54.529444,25.634444,NaN,Vilnius,NaN,NaN,NaN,NaN,Aukštojas Hill is the highest point in Lithuan...


In [13]:
# as an example of a duplicate
Peak_df.loc['Aukstojas']

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Aukstojas,"May, Jun, Jul, Aug, Sep",Europe,Vilnius,Lithuania,Walk up,961.0,293.0,NaN,54.529444,25.634444,NaN,Vilnius,NaN,NaN,NaN,NaN,Aukštojas Hill is the highest point in Lithuan...
Aukstojas,"May, Jun, Jul, Aug, Sep",Europe,Vilnius,Lithuania,Walk up,961.0,293.0,NaN,54.529444,25.634444,NaN,Vilnius,NaN,NaN,NaN,NaN,Aukštojas Hill is the highest point in Lithuan...


In [14]:
# removing the duplicates
Peak_df.drop_duplicates(keep='first', inplace=True)

In [15]:
# making sure duplicates were removed
Peak_df[Peak_df.duplicated()==True]

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:


In [16]:
Peak_df.dtypes

Best months for climbing:        object
Continent:                       object
Convenient Center:               object
Country:                         object
Difficulty:                      object
Elevation (feet):               float64
Elevation (meters):             float64
First successful climber(s):     object
Latitude:                       float64
Longitude:                      float64
Most recent eruption:            object
Nearest major airport:           object
Province:                        object
Range/Region:                    object
State:                           object
Volcanic status:                 object
Year first climbed:              object
dtype: object

In [17]:
# saving to file dataframe with no duplicates
Peak_df.to_csv('Peak_data_db.csv')

### Checking dtypes are reasonable

In [18]:
# read the data
Peak_df = pd.read_csv('Peak_data_db.csv', index_col=0)

In [19]:
Peak_df.dtypes

Best months for climbing:        object
Continent:                       object
Convenient Center:               object
Country:                         object
Difficulty:                      object
Elevation (feet):               float64
Elevation (meters):             float64
First successful climber(s):     object
Latitude:                       float64
Longitude:                      float64
Most recent eruption:            object
Nearest major airport:           object
Province:                        object
Range/Region:                    object
State:                           object
Volcanic status:                 object
Year first climbed:              object
dtype: object

#### Seems to be alright

### Put None where data is missing

In [9]:
# read the data
Peak_df = pd.read_csv('Peak_data_db.csv', index_col=0)

In [10]:
Peak_df.dtypes

Best months for climbing:        object
Continent:                       object
Convenient Center:               object
Country:                         object
Difficulty:                      object
Elevation (feet):               float64
Elevation (meters):             float64
First successful climber(s):     object
Latitude:                       float64
Longitude:                      float64
Most recent eruption:            object
Nearest major airport:           object
Province:                        object
Range/Region:                    object
State:                           object
Volcanic status:                 object
Year first climbed:              object
dtype: object

In [22]:
Peak_df.isna()

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Mount Stuart,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False
Mount Irvine,True,False,False,False,True,False,False,True,False,False,True,True,True,False,False,True,True
Peak 7480,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,True,True
Norton Peak,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,True,True
Mont Aiguille,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,True,False
Aiguille Verte,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,True,False
Mount Alberta,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False
Aletschhorn,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,True,False
Alpamayo,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,True,False
Ama Dablam,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,True,False


In [23]:
Peak_df.fillna(value='None', inplace=True)

In [24]:
Peak_df

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Mount Stuart,"Jun, Jul, Aug",North America,"Leavenworth, Washington",United States,Technical Climb,9415,2869,A. McPherson and party,47.4752,-120.902,None,"Seattle, Washington",None,Cascade Range,Washington,None,1873.0
Mount Irvine,None,North America,Lone Pine,United States,None,13914,4241,None,36.5558,-118.262,None,None,None,Sierra Nevada,California,None,None
Peak 7480,"Jan, Feb, Apr, May, Sep, Oct",North America,"Carlsbad, NM.",United States,Walk up,7480,2280,None,32,-104.817,None,"El Paso, TX.",None,Southwest Basins and Ranges,Texas,None,None
Norton Peak,"Jun, Jul, Aug, Sep, Oct",North America,Ketchum,United States,Scramble,10336,3150,None,43.7619,-114.652,None,Hailey,None,Idaho-Bitterroot Rockies,Idaho,None,None
Mont Aiguille,"Jul, Aug, Sep",Europe,"Grenoble, France",France,Technical Climb,6842,2085,Antoine de Ville,44.8333,5.55,None,"Grenoble or Lyon, France",None,Dauphine Alps,None,None,1492
Aiguille Verte,"Jul, Aug, Sep",Europe,"Chamonix, France",France,Basic Snow/Ice Climb,13524,4122,"Edward Whymper, Christian Almer, Franz Biener",45.9833,6.43333,None,"Chamonix, France",None,Savoy Alps,None,None,1865
Mount Alberta,"Jun, Jul, Aug, Sep",North America,"Jasper, Alberta",Canada,Technical Climb,11874,3619,"Yuko Maki and party with , H. Fuhrer, H. Kohle...",52.288,-117.476,None,"Edmonton or Calgary, Alberta",Alberta,Canadian Rockies,None,None,1925
Aletschhorn,"Jul, Aug, Sep",Europe,"Bettmeralp or Blatten, Switzerland",Switzerland,Technical Climb,13763,4195,"Francis Tuckett, J.J. Bennen, P. Bohren, Victo...",46.4667,7.98333,None,"Bern, Switzerland",None,Berner Oberland,None,None,1859
Alpamayo,"May, Jun, Jul, Aug, Sep",South America,"Huarez, Peru",Peru,Basic Snow/Ice Climb,19511,5947,"G. Hauser, F. Knauss, B. Huhn, H. Wiedmann (Ge...",-8.8782,-77.6544,None,"Lima, Peru",None,Peruvian Andes,None,None,1951
Ama Dablam,"Apr, May, Sep, Oct",Asia,"Namche Bazar, Nepal",Nepal,Major Mountain Expedition,22494,6856,"Mike Gill, Barry Bishop, Mike Ward, Wally Romanes",27.8611,86.8611,None,"Kathmandu, Nepal",None,Central Nepal Himalaya,None,None,1961


In [25]:
Peak_df.dtypes

Best months for climbing:       object
Continent:                      object
Convenient Center:              object
Country:                        object
Difficulty:                     object
Elevation (feet):               object
Elevation (meters):             object
First successful climber(s):    object
Latitude:                       object
Longitude:                      object
Most recent eruption:           object
Nearest major airport:          object
Province:                       object
Range/Region:                   object
State:                          object
Volcanic status:                object
Year first climbed:             object
dtype: object

#### as we can see above, replacing NaN with None changed dtype of float64 columns to object
#### I will not save the changes to file because of that and keep the default NaN

In [34]:
# # saving to file dataframe with None replacing NaN
# Peak_df.to_csv('Peak_data_db.csv')

### Correcting Elevations

* If only one of the elevations is given, then fill the missing data.
* If both elevations are missing, then drop the peak record.

In [2]:
# read the data
Peak_df = pd.read_csv('Peak_data_db.csv', index_col=0)

In [3]:
Peak_df.dtypes

Best months for climbing:        object
Continent:                       object
Convenient Center:               object
Country:                         object
Difficulty:                      object
Elevation (feet):               float64
Elevation (meters):             float64
First successful climber(s):     object
Latitude:                       float64
Longitude:                      float64
Most recent eruption:            object
Nearest major airport:           object
Province:                        object
Range/Region:                    object
State:                           object
Volcanic status:                 object
Year first climbed:              object
dtype: object

In [6]:
# find out if missing elevations exist
Peak_df[Peak_df[('Elevation (feet):')].isna()]

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Monte Olivia,"Jan, Feb, Mar, Nov, Dec",South America,Ushuaia,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ushuaia,NaN,Central Argentina-Chile,NaN,NaN,NaN
Nina Peak,"Apr, May, Jun, Jul, Aug, Sep, Oct, Nov",Australia/Oceania,NaN,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Townsville,NaN,NaN,NaN,NaN,NaN
Desperation Peak,"May, Jun, Jul, Aug, Sep",North America,Santa Fe,United States,Walk up,NaN,NaN,NaN,NaN,NaN,NaN,Santa Fe,NaN,NaN,New Mexico,NaN,NaN
Huckleberry Point,"Jul, Aug",North America,Bracebridge,Canada,Walk up,NaN,NaN,First Nations,NaN,NaN,NaN,Toronto Pearson Airport,Ontario,Canadian Shield,NaN,NaN,NaN
Leyko Oros,Dec,Europe,West Alpis,Greece,Technical Climb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Grammos,Dec,Europe,NaN,Greece,NaN,NaN,NaN,NaN,40.348333,20.779444,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# find out if missing elevations exist
Peak_df[Peak_df[('Elevation (meters):')].isna()]

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Monte Olivia,"Jan, Feb, Mar, Nov, Dec",South America,Ushuaia,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ushuaia,NaN,Central Argentina-Chile,NaN,NaN,NaN
Nina Peak,"Apr, May, Jun, Jul, Aug, Sep, Oct, Nov",Australia/Oceania,NaN,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Townsville,NaN,NaN,NaN,NaN,NaN
Desperation Peak,"May, Jun, Jul, Aug, Sep",North America,Santa Fe,United States,Walk up,NaN,NaN,NaN,NaN,NaN,NaN,Santa Fe,NaN,NaN,New Mexico,NaN,NaN
Huckleberry Point,"Jul, Aug",North America,Bracebridge,Canada,Walk up,NaN,NaN,First Nations,NaN,NaN,NaN,Toronto Pearson Airport,Ontario,Canadian Shield,NaN,NaN,NaN
Mount Guera,Dec,Africa,NDJAMENA,Chad,Walk up,4750.0,NaN,NaN,11.916700,18.200000,NaN,NDJAMENA,NaN,NaN,NaN,NaN,NaN
Leyko Oros,Dec,Europe,West Alpis,Greece,Technical Climb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Grammos,Dec,Europe,NaN,Greece,NaN,NaN,NaN,NaN,40.348333,20.779444,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Burney Mountain,NaN,North America,Burney,United States,Walk up,7863.0,NaN,NaN,40.806569,-121.627931,NaN,Redding,NaN,Cascade Range,California,NaN,NaN
Mount Torbert,"Apr, May, Jun, Jul, Aug, Sep, Oct",North America,Anchorage,United States,Major Mountain Expedition,11413.0,NaN,NaN,61.408600,-152.412000,NaN,Anchorage International,NaN,Alaska Range,Alaska,NaN,NaN


In [35]:
# # find out if missing elevations exist
# Peak_df[(Peak_df[('Elevation (feet):')]=='NaN') | (Peak_df[('Elevation (meters):')]=='NaN')]

In [9]:
Peak_df['Elevation (meters):'] = 0.3048 * Peak_df['Elevation (feet):']

In [11]:
# check again for missing values to see if it was calculated
Peak_df[Peak_df[('Elevation (meters):')].isna()]

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Monte Olivia,"Jan, Feb, Mar, Nov, Dec",South America,Ushuaia,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ushuaia,NaN,Central Argentina-Chile,NaN,NaN,NaN
Nina Peak,"Apr, May, Jun, Jul, Aug, Sep, Oct, Nov",Australia/Oceania,NaN,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Townsville,NaN,NaN,NaN,NaN,NaN
Desperation Peak,"May, Jun, Jul, Aug, Sep",North America,Santa Fe,United States,Walk up,NaN,NaN,NaN,NaN,NaN,NaN,Santa Fe,NaN,NaN,New Mexico,NaN,NaN
Huckleberry Point,"Jul, Aug",North America,Bracebridge,Canada,Walk up,NaN,NaN,First Nations,NaN,NaN,NaN,Toronto Pearson Airport,Ontario,Canadian Shield,NaN,NaN,NaN
Leyko Oros,Dec,Europe,West Alpis,Greece,Technical Climb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Grammos,Dec,Europe,NaN,Greece,NaN,NaN,NaN,NaN,40.348333,20.779444,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
Peak_df.dtypes

Best months for climbing:        object
Continent:                       object
Convenient Center:               object
Country:                         object
Difficulty:                      object
Elevation (feet):               float64
Elevation (meters):             float64
First successful climber(s):     object
Latitude:                       float64
Longitude:                      float64
Most recent eruption:            object
Nearest major airport:           object
Province:                        object
Range/Region:                    object
State:                           object
Volcanic status:                 object
Year first climbed:              object
dtype: object

In [13]:
# saving to file dataframe after correcting missing Elevations
Peak_df.to_csv('Peak_data_db.csv')

### Data manipulation
* If a peak is listed with more than a single country, then use the first country.


In [14]:
# read the data
Peak_df = pd.read_csv('Peak_data_db.csv', index_col=0)

In [15]:
# checking if there are peaks with more than one country
Peak_df[Peak_df['Country:'].str.contains('/', na=False)]

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:
Balaitous,"Jun, Jul, Aug, Sep",Europe,"Spain: Jaca, Sabiñanigo. France: Arrens",Spain/France,Basic Snow/Ice Climb,10322.0,3146.1456,"Peytier, Hossard and party",42.838717,-0.290966,NaN,"Lourdes or Toulouse, France, Zaragoza, Pamplon...",NaN,Pyrenees,NaN,NaN,1825
Bernina,"Jul, Aug, Sep",Europe,"St. Moritz, Switzerland",Switzerland/Italy,Basic Snow/Ice Climb,13284.0,4048.9632,"Johann Coaz, Jon Ragut Tschamer, Lorenz Ragut ...",46.382393,9.908240,NaN,"Zurich, Switzerland",NaN,Rhaetian Alps,NaN,NaN,1850
Chomolhari,"Apr, May, Sep, Oct",Asia,"Thimpu, Bhutan",Bhutan/China,Major Mountain Expedition,23997.0,7314.2856,Spenser Chapman and Sherpa porter Pasang Dawa ...,27.833300,89.266700,NaN,"Paro, Bhutan",NaN,Bhutan Himalaya,NaN,NaN,1937
Mount Elgon,"Jan, Feb, Jun, Jul, Aug, Dec",Africa,"Kitale, Kenya",Uganda/Kenya,Walk up,14178.0,4321.4544,Kmunke and Stigler,1.133330,34.550000,NaN,"Nairobi, Kenya",NaN,Central East Africa Plateau,NaN,Extinct,1911
Mount Everest,"Apr, May",Asia,"Namche Bazar, Nepal",Nepal/China,Major Mountain Expedition,29035.0,8849.8680,"Edmund Hillary, Tenzing Norgay",27.988100,86.925300,NaN,"Kathmandu, Nepal",NaN,Central Nepal Himalaya,NaN,NaN,1953
Ganesh Himal,"Apr, May, Sep, Oct",Asia,"Kathmandu, Nepal",Nepal/China,Major Mountain Expedition,24373.0,7428.8904,"Raymond Lambert, Claude Kogan, E. Gauchat",28.391551,85.126963,NaN,"Kathmandu, Nepal",NaN,Central Nepal Himalaya,NaN,NaN,1955
Gyachung Kang,"Apr, May",Asia,"Kathmandu, Nepal",Nepal/China,Major Mountain Expedition,26089.0,7951.9272,"Y. Kato, K. Sakaizawa, Pasang Phutar, (Japanes...",28.100000,86.750000,NaN,"Kathmandu, Nepal",NaN,Central Nepal Himalaya,NaN,NaN,1964
K2,"Jun, Jul, Aug",Asia,"Skardu, Pakistan",Pakistan/China,Major Mountain Expedition,28253.0,8611.5144,"Achille Compagnoni, Lino Lacedelli (Italian ex...",35.882500,76.513300,NaN,"Islamabad, Pakistan",NaN,Karakoram,NaN,NaN,1954
Kamet,"Jul, Aug, Sep",Asia,"Hardwar, India",India/China,Major Mountain Expedition,25447.0,7756.2456,"Frank Smythe, Eric Shipton, R.L. Holdsworth, L...",30.920278,79.593611,NaN,"Delhi, India",NaN,Garhwal Himalaya,NaN,NaN,1931
Kangchenjunga,"Apr, May",Asia,"Hille, Nepal via Dharan Bazar",Nepal/India,Major Mountain Expedition,28169.0,8585.9112,"George Band, Joe Brown, (British expedition)",27.702500,88.148300,NaN,"Kathmandu, Nepal",NaN,Sikkim-Eastern Nepal Himalaya,NaN,NaN,1955


In [16]:
# in order to avoid next line error because it cannot process NaN values
Peak_df['Country:'].fillna(value='None', inplace=True)

In [20]:
Peak_df[Peak_df['Country:'].isna()]

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:


In [23]:
#removing the second country from cells where it exists
Peak_df['Country:'] = Peak_df['Country:'].str.split('/').str.get(0)

In [25]:
# checking again if more than one country is in 'Country: column
# in order to see if switch worked or not
Peak_df[Peak_df['Country:'].str.contains('/', na=False)]

,Best months for climbing:,Continent:,Convenient Center:,Country:,Difficulty:,Elevation (feet):,Elevation (meters):,First successful climber(s):,Latitude:,Longitude:,Most recent eruption:,Nearest major airport:,Province:,Range/Region:,State:,Volcanic status:,Year first climbed:


In [26]:
# saving to file dataframe second country removed
Peak_df.to_csv('Peak_data_db.csv')

### Data manipulation (continued)
* If the date of the first climbing is recorded as an irregular date, then try to manipulate it so that only the year data is preserved. If the data is too obscure, then put None instead.
* Note that some terms are written differently for different peaks. Use the lower() method to avoid mistakes.

In [2]:
# read the data
Peak_df = pd.read_csv('Peak_data_db.csv', index_col=0)

In [9]:
#Peak_df['Year first climbed:']

In [10]:
# pd.to_datetime(Peak_df['Year first climbed:'],
#                yearfirst=True, format='%Y', errors='coerce').dt.to_period('Y')

In [13]:
Peak_df['Year first climbed:'] = pd.to_datetime(
    Peak_df['Year first climbed:'], errors='coerce').dt.year

In [12]:
# pd.to_datetime(Peak_df['Year first climbed:'],
#                errors='coerce').dt.strftime('%Y')

In [11]:
#Peak_df['Year first climbed:'].str.split()#contains('\d\d\d\d',regex=True, na=False)]

Mount Stuart           1873.0
Mount Irvine              NaN
Peak 7480                 NaN
Norton Peak               NaN
Mont Aiguille             NaN
Aiguille Verte         1865.0
Mount Alberta          1925.0
Aletschhorn            1859.0
Alpamayo               1951.0
Ama Dablam             1961.0
Amne Machin            1981.0
Aniakchak                 NaN
Ancohuma               1915.0
Annapurna              1950.0
Api                    1960.0
Argentera              1879.0
Mount Asgard           1953.0
Askja                     NaN
Mount Aspiring         1909.0
Mount Assiniboine      1901.0
Mount Athabasca        1898.0
Atitlan                   NaN
Ausangate              1953.0
Mount Baker            1906.0
Mount Baker            1868.0
Balaitous              1825.0
Ball's Pyramid         1965.0
Hyndman Peak              NaN
Gora Belukha           1914.0
Ben Nevis                 NaN
                        ...  
Pointe Percée             NaN
Cousins Peak              NaN
Cumbre San

In [15]:
# saving to file dataframe second country removed
Peak_df.to_csv('Peak_data_db.csv')